# import

In [1]:
import pprint

from dotenv import load_dotenv

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage



# loading env variables

In [2]:
load_dotenv()

True

# Langain & Ollama

## check ollama status

In [3]:
!curl --location 'http://127.0.0.1:11434/api/generate' \
--header 'Content-Type: application/json' \
--data '{ \
    "model": "llama3:8b", \
    "prompt": "why is the sky blue ?",  \
    "stream": false, \
    "options": { \
        "temperature": 0 \
    } \
}' \
# | python -m json.tool

{"model":"llama3:8b","created_at":"2024-05-03T21:44:45.32952837Z","response":"What a great question!\n\nThe short answer is: scattering.\n\nHere's a more detailed explanation:\n\nWhen sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light, so they scatter the shorter wavelengths of light more efficiently. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.\n\nThe shorter wavelengths of light, like blue and violet, are scattered more than the longer wavelengths, like red and orange. This is because the smaller molecules are better at interacting with the shorter wavelengths. As a result, the blue light is distributed throughout the atmosphere, giving the sky its blue appearance.\n\nHere's a rough breakdown of what happens:\n\n1. Sunlight enters the atmosphere, containing all th

In [5]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"llama3:8b","model":"llama3:8b","modified_at":"2024-05-03T23:43:53.896559863+02:00","size":4661224578,"digest":"a6990ed6be412c6a217614b0ec8e9cd6800a743d5dd7e1d7fbe9df09e61d5615","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8B","quantization_level":"Q4_0"}}]}

## Ollama model: configuration

In [6]:
llm = Ollama(model="llama3:8b", temperature=0)

## testing llm: invoke


In [7]:
# llm.invoke(input="tell me a joke")
response = llm.invoke("hello ollama!")
# response = llm.invoke("Create an agent that uses Ollama function calling in Langchain.")

print(response)

Hello there! It's great to meet you, friend! I'm Ollama, your friendly AI companion. What brings you here today? Do you have a question, need some help or just want to chat? I'm all ears (or rather, all text)!


## testing llm: chat prompt template

In [8]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

chain = chat_prompt_template | llm

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'm excited to share some insights on how Langsmith can assist with testing.

Langsmith's AI-powered language processing capabilities can be leveraged in various ways to enhance the testing process. Here are some ideas:

1. **Automated Testing**: Langsmith can help automate testing by generating test cases based on natural language descriptions. This can save time and reduce the likelihood of human error.
2. **Test Data Generation**: Langsmith's AI can generate test data, such as input values or expected output results, based on predefined patterns or scenarios. This can be particularly useful for testing complex systems or APIs.
3. **Error Detection**: Langsmith's language processing capabilities can help detect errors in test cases by analyzing the syntax and semantics of the test code. This can identify potential issues before they cause problems during testing.
4. **Test Case Prioritization**: Langsmith can analyze test cases based o

## testing llm: chat prompt template & StrOutputParser

In [24]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

output_parser = StrOutputParser()

chain = chat_prompt_template | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

As a world-class technical documentation writer, I'm excited to share some insights on how Langsmith can assist with testing.

Langsmith is an AI-powered language translation tool that can be leveraged for testing purposes in several ways:

1. **Automated Testing**: Langsmith's API integration allows you to automate testing by generating test cases and scenarios in multiple languages. This ensures that your software or application functions correctly across different linguistic and cultural contexts.
2. **Localization Testing**: Langsmith can help with localization testing by providing translated test data, which enables you to verify the accuracy of translations and ensure that your product is culturally relevant for target markets.
3. **Error Detection**: By generating test cases in multiple languages, Langsmith helps detect errors or inconsistencies in your software's translation, allowing you to identify and fix issues before they impact users.
4. **Test Data Generation**: Langsmit

## create vector store & create a retriever

In [25]:
# 1. select a specfic datasource. In this case a web page. 
# 2. save extracted content from the web page as docs.
# 3. index the docs using FAISS vector store.
# 4. convert the vector store to retriever.

web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = web_base_loader.load()
# print(docs)
# print(type(docs))
# print(len(docs))
# type(docs[0])
# print(docs[0].page_content)

recursive_character_text_splitter = RecursiveCharacterTextSplitter()
documents = recursive_character_text_splitter.split_documents(documents=docs)

# print(type(documents))
# print(documents)
# print(len(documents))
# print(documents[0])
# print(documents[3])

ollama_embedding = OllamaEmbeddings(model="llama3:8b")
vector_store = FAISS.from_documents(documents=documents, embedding=ollama_embedding)

# vector_store.index.ntotal
# vector_store._get_retriever_tags()
# vector_store.index_to_docstore_id
# type(vector_store.index_to_docstore_id)

vector_store_retriever = vector_store.as_retriever()

## document chain

In [26]:
# 5. create a chat prompt template
# 6. create a stuff document chain that accepts a llm model and chat prompt template & we can also run stuff document chain by passing in documents directly

chat_prompt_template = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

documents_chain = create_stuff_documents_chain(llm=llm, prompt=chat_prompt_template)
response = documents_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": documents        
    }
)
print(response)

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Creating datasets**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. **Running custom evaluations**: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results.
3. **Comparison view**: Langsmith provides a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.
4. **Playground environment**: The playground environment allows developers to quickly test out different prompts and models, with each run logged in the system and able to be used to create test cases or compare with other runs.
5. **Beta testing**: LangSmith supports beta testing by allowing developers to collect more data on how their LLM applications are performing in real-world scenarios, including feedback colle

## retrieval chain

In [27]:
# 7. create a document retrieval chain that takes vector store retriever and stuff document chain

retrieval_chain = create_retrieval_chain(vector_store_retriever, documents_chain) 
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

# print(type(response))
pprint.pprint(response, indent=4)

{   'answer': 'Based on the provided context, LangSmith can help with testing '
              'in several ways:\n'
              '\n'
              '1. **Tracing**: LangSmith allows developers to create datasets, '
              'which are collections of inputs and reference outputs, and use '
              'these to run tests on their LLM applications.\n'
              '2. **Evaluation**: LangSmith provides native rendering of chat '
              'messages, functions, and retrieve documents, making it easy to '
              'evaluate the performance of an application.\n'
              '3. **Comparison View**: Langsmith offers a comparison view for '
              'test runs, allowing developers to track and diagnose '
              'regressions in test scores across multiple revisions of their '
              'application.\n'
              '4. **Playground**: The playground environment allows rapid '
              'iteration and experimentation, enabling developers to quickly '
    

## conversation retrieval chain

In [49]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

history_aware_retriever_chain = create_history_aware_retriever(llm, vector_store_retriever, chat_prompt_template)

In [47]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, chat_prompt_template)
retrieval_chain = create_retrieval_chain(history_aware_retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "tell me how"
})

pprint.pprint(response)

{'answer': 'LangSmith is designed to support various workflows throughout the '
           "development lifecycle of your LLM application. Here's an overview "
           'of how Langsmith can help you test your LLM applications:\n'
           '\n'
           '1. **Prototyping**: Langsmith provides a playground environment '
           'for rapid iteration and experimentation. You can quickly test out '
           'different prompts and models, and each run is logged in the '
           'system.\n'
           '2. **Debugging**: When things go wrong, Langsmith gives you clear '
           'visibility and debugging information at each step of an LLM '
           'sequence, making it easier to identify and root-cause issues.\n'
           '3. **Initial Test Set**: You can create datasets, which are '
           'collections of inputs and reference outputs, and use these to run '
           'tests on your LLM applications.\n'
           '4. **Comparison View**: Langsmith allows you to view